In [ ]:
from google.colab import drive  #for colab only
drive.mount('/content/gdrive')

In [ ]:
import librosa
import librosa.display
import os
import numpy as np
import math
import re

## Load DataSet Files name

In [ ]:
#torgoPath = r"E:\tergo dataset\data and labels" # DataSet path update it 
UASpeech = r"data_C" # DataSet path update it 

In [ ]:
def readDataSetFiles(DataSetPath,labels=True,mark='\\'):
    allFilesInPath=sorted(os.listdir(DataSetPath))
    dataSetAllMainFolders=[File for File in allFilesInPath if os.path.isdir(DataSetPath+mark+File) ]  #[f_audio,f_label,m_audio,m_label]
    allDataFilesInDataSetPaths =[DataSetPath +mark + Folder+mark+File for Folder in dataSetAllMainFolders for File in sorted(os.listdir(DataSetPath +mark + Folder)) if re.search('_audio',Folder)] #get all wav files in one list
    if labels:
        allLabelFilesInDataSetPaths=[DataSetPath +mark+ Folder+mark+File for Folder in dataSetAllMainFolders for File in sorted(os.listdir(DataSetPath + mark + Folder)) if re.search('_label',Folder)] #get all txt files in one list
    else:allLabelFilesInDataSetPaths=[]
    del(dataSetAllMainFolders)
    return allDataFilesInDataSetPaths,allLabelFilesInDataSetPaths


In [ ]:
allDataFilesInDataSetPaths,allLabelFilesInDataSetPaths=readDataSetFiles(UASpeech,labels=True,mark='/')

In [ ]:
print(len(allDataFilesInDataSetPaths))
print(len(allLabelFilesInDataSetPaths))
print('first data file',allDataFilesInDataSetPaths[:1],'last data file',allDataFilesInDataSetPaths[-1:])
print('first label file',allLabelFilesInDataSetPaths[:1],'last label file',allLabelFilesInDataSetPaths[-1:])

In [ ]:
def readLabelsLinesFromTxt(labelsFiles):
    errorReadLabelTxtFile,allLabelFilesInDataSetPaths=[],[]
    for labelsFileName in labelsFiles:
        try:
            with open(labelsFileName) as labelTxtFile:
                allLabelsInData=sorted(labelTxtFile.readlines())
                allLabelsInData=[line.split(':')[-1].strip() for line in allLabelsInData]
        except:
            errorReadLabelTxtFile.append(labelsFileName)
    return allLabelsInData,errorReadLabelTxtFile

In [ ]:
%%time
allLabelsInData,errorReadLabelTxtFile=readLabelsLinesFromTxt(allLabelFilesInDataSetPaths)

In [ ]:
print(len(allLabelsInData))
print(len(errorReadLabelTxtFile))
print(allLabelsInData[:10])
print(allLabelsInData[-10:])

In [ ]:
# print(allDataFilesInDataSetPaths[38250])
# print(allLabelsInData[38250])

In [ ]:
# allDataFilesInDataSetPaths=allDataFilesInDataSetPaths[49405:]
# allLabelsInData=allLabelsInData[49405:]
# allDataFilesInDataSetPaths=allDataFilesInDataSetPaths[:38250]
# allLabelsInData=allLabelsInData[:38250]

In [ ]:
# print(allDataFilesInDataSetPaths[:5])
# print(allLabelsInData[:5])

# set variables as default values 

In [ ]:
SAMPLE_RATE = 16000
AUDIO_DURATION = 10 # measured in seconds


# Extract signal and mfcc feature form waves

In [ ]:
def extractSignal_mfccFeature(allAudioFilesPaths,sampleRate,audioDuration):
    allWavesInDataAsNumpyArray, errorFilesInDataset,mfccFeaturesNumpyArray= [] ,[] ,[] 
    for dataFilePath in allAudioFilesPaths: # all wave files paths
        print(dataFilePath.split('\\')[-1])
        try:
            signal, sampleRate = librosa.load(dataFilePath , sr = sampleRate , duration = audioDuration , res_type='kaiser_fast')#get signals and sampleRate in ane wave (dataFilePath)   
            mfccs = np.mean(librosa.feature.mfcc(y=signal, sr=sampleRate, n_mfcc=40).T,axis=0)  #get mfcc array featture for one wave   
        except :
            errorFilesInDataset.append(dataFilePath) #collect name of file that have error to find it
        feature = np.array(mfccs).reshape([-1,1]) #get mfcc array featture for one wave after reshape it
        mfccFeaturesNumpyArray.append(feature)  #apeend featture array to get list of all waves feature                       
        allWavesInDataAsNumpyArray.append(signal) #apeend signal array to get list of all waves signals
    return mfccFeaturesNumpyArray,allWavesInDataAsNumpyArray,errorFilesInDataset

In [ ]:
%%time
mfccFeaturesAsNumpyArray,allWavesInDataAsNumpyArray,errorFilesInDataset = extractSignal_mfccFeature(allDataFilesInDataSetPaths[:],SAMPLE_RATE,AUDIO_DURATION)

In [ ]:
# def extractTextFromTextFiles(LabelOfDataSetPaths):
#     allLabelsInDatasetText,errorTextFilePaths=[],[]
#     for labelFilePath in LabelOfDataSetPaths:  # all txt files
#         try:
#             with open(labelFilePath, "r") as labelTextFile: # open label file
#                 allLabelsInDatasetText.append(labelTextFile.read().strip()) #get labels text in one list without duple spaces or \n
#             print(labelFilePath.split('\\')[-1])
#         except:
#             errorTextFilePaths.append(labelFilePath)
#             print(errorTextFilePaths[:10])
#     return allLabelsInDatasetText,errorTextFilePaths

In [ ]:
print(allLabelsInData[:10])
print(len(allLabelsInData),'\n\n')
print(allWavesInDataAsNumpyArray[0])
print(len(allWavesInDataAsNumpyArray),'\n\n')
print(errorFilesInDataset)

In [ ]:
print(mfccFeaturesAsNumpyArray[0])
print(len(mfccFeaturesAsNumpyArray))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12,4))
librosa.display.waveplot(allWavesInDataAsNumpyArray[0], sr = SAMPLE_RATE )

In [ ]:
MFCCs = librosa.feature.mfcc(allWavesInDataAsNumpyArray[0], SAMPLE_RATE, n_mfcc=13)
# display MFCCs
plt.figure(figsize=(10,4))
librosa.display.specshow(MFCCs, x_axis = 'time')
plt.colorbar()
plt.title("MFCCs")
plt.tight_layout()
plt.show()

In [ ]:
MFCCs = librosa.feature.mfcc(allWavesInDataAsNumpyArray[0], SAMPLE_RATE, n_mfcc=13)
MFCCs = MFCCs.T
print(MFCCs)

In [ ]:
print(mfccFeaturesAsNumpyArray[0])

In [ ]:
  X = np.array(mfccFeaturesAsNumpyArray)
  print(X.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(allLabelsInData)
classes = len(list(le.classes_))
print(y.shape)
print(classes)

In [ ]:
from keras.utils import np_utils
y = np_utils.to_categorical(y)
#, num_classes = len(allLabelsInDatasetText)
y = np.array(y)  
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state = 42)

# x_test,x_val, y_test,y_val = train_test_split(x_test, y_test, test_size= 0.3, random_state = 42)

In [ ]:
# print(len(x_train))
# print(len(y_train))
# print(len(x_test))
# print(len(y_test))
# print(len(x_val))
# print(len(y_val))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , Activation , Flatten , Conv2D, MaxPooling2D, TimeDistributed, LSTM , Bidirectional ,SpatialDropout2D,SpatialDropout1D
from keras.optimizers import Adam
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
print(x_train.shape)
print (x_test.shape)
print (y_train.shape)

In [ ]:
from keras.layers import LSTM, TimeDistributed, Dense, Activation, Input 
from keras.optimizers import Adam
from numpy import zeros
import keras.layers as KL
import keras.models as KM
import keras.engine as KE
# from keras_ctcmodel.CTCModel import CTCModel as CTCModel
from tensorflow.keras.layers import BatchNormalization

### for using GPU but must be nevdia not amd

In [ ]:
import keras
import tensorflow as tf

config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.compat.v1.Session(config=config) 
keras.backend.set_session(sess)

from tensorflow.python.client import device_lib    #for print the resources of device
print(device_lib.list_local_devices())  

In [ ]:
inputshape = x_train.shape[1:]
model = Sequential()

# using BLSTM
model.add(Bidirectional(LSTM(512,return_sequences=True),input_shape = x_train.shape[1:] ))
model.add(SpatialDropout1D(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(512,return_sequences=True)))
model.add(SpatialDropout1D(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(512,return_sequences=True)))
model.add(SpatialDropout1D(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(512,return_sequences=True)))
model.add(SpatialDropout1D(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(512)))#recurrent_dropout=.5,recurrent_activation='relu'
model.add(Dropout(0.3))
model.add(BatchNormalization())

#fully connected
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
#softmax
model.add(Dense(classes,activation = 'softmax'))

model.summary()

In [ ]:
# def cc_loss_w_smoothing(y_true, y_pred):
#     label_smoothing = 0.1
#     return keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=label_smoothing )

In [ ]:
model.compile(optimizer= Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
History = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=80,batch_size=256)

In [ ]:
History.history

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
model.save('/content/gdrive/MyDrive/Bidirectional_LSTM_edit/Bidirectional_LSTM_dataM  76%-43%')

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test loss, test acc:", results)

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predicted = model.predict(x_test[:3])
print("True=%s, Predicted=%s" % (x_test[:3], predicted[:3]))

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    """
    plots the confusion matrix
    """
    matrix = confusion_matrix(y_true, y_pred)    
    fig, ax = plt.subplots(figsize=(12,10))
    plt.imshow(matrix)
    max_confusions = 0
    confused_classes = (-1, -1)
    for i, true_label in enumerate(matrix):
        for j, predicted_label in enumerate(true_label):
            text = ax.text(j, i, matrix[i, j],
                        ha="center", va="center", color="w");
    plt.tick_params(axis=u'both', which=u'both',length=0)
    plt.title("Confusion Matrix");

def plot_loss(history):
    from matplotlib import pyplot as plt
    plt.figure(figsize=(10,7))
    plt.plot(history.history['loss'], c ="darkblue")
    plt.plot(history.history['val_loss'], c ="crimson")
    plt.legend(["Train", "Validation"])
    plt.title("Model loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True, alpha = 0.2)
    plt.show()

In [ ]:
plot_loss(History)